# Lab 4-2: Load Data

Author: Seungjae Lee (이승재)

<div class="alert alert-warning">
    We use elemental PyTorch to implement linear regression here. However, in most actual applications, abstractions such as <code>nn.Module</code> or <code>nn.Linear</code> are used.
</div>

## Slicing 1D Array

In [1]:
nums = [0, 1, 2, 3, 4]

In [2]:
print(nums)

[0, 1, 2, 3, 4]


index 2에서 4 전까지 가져와라. (앞 포함, 뒤 비포함)

In [3]:
print(nums[2:4])

[2, 3]


index 2부터 다 가져와라.

In [4]:
print(nums[2:])

[2, 3, 4]


index 2 전까지 가져와라. (역시 뒤는 비포함)

In [5]:
print(nums[:2])

[0, 1]


전부 가져와라

In [6]:
print(nums[:])

[0, 1, 2, 3, 4]


마지막 index 전까지 가져와라. (뒤는 비포함!)

In [7]:
print(nums[:-1])

[0, 1, 2, 3]


assign 도 가능!

In [8]:
nums[2:4] = [8, 9]

In [9]:
print(nums)

[0, 1, 8, 9, 4]


## Slicing 2D Array

In [10]:
import numpy as np

In [11]:
b = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])

In [12]:
print(b)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


In [13]:
b[:, 1]

array([ 2,  6, 10])

In [14]:
b[-1]

array([ 9, 10, 11, 12])

In [15]:
b[-1, :]

array([ 9, 10, 11, 12])

In [16]:
b[-1, ...]

array([ 9, 10, 11, 12])

In [17]:
b[0:2, :]

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

## Loading Data from `.csv` file

In [18]:
import numpy as np

In [19]:
xy = np.loadtxt('data-01-test-score.csv', delimiter=',', dtype=np.float32)

In [20]:
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

In [21]:
print(x_data.shape)
print(len(x_data))
print(x_data[:5])

(25, 3)
25
[[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]]


In [22]:
print(y_data.shape)
print(len(y_data))
print(y_data[:5])

(25, 1)
25
[[152.]
 [185.]
 [180.]
 [196.]
 [142.]]


## Imports

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [24]:
torch.manual_seed(1)

## Low-level Implementation

In [25]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    hypothesis = x_train.matmul(W) + b
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} Cost {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost 26811.960938
Epoch    1/20 Cost 9920.529297
Epoch    2/20 Cost 3675.298828
Epoch    3/20 Cost 1366.261108
Epoch    4/20 Cost 512.542236
Epoch    5/20 Cost 196.896484
Epoch    6/20 Cost 80.190910
Epoch    7/20 Cost 37.038647
Epoch    8/20 Cost 21.081354
Epoch    9/20 Cost 15.178741
Epoch   10/20 Cost 12.993667
Epoch   11/20 Cost 12.183028
Epoch   12/20 Cost 11.880545
Epoch   13/20 Cost 11.765955
Epoch   14/20 Cost 11.720857
Epoch   15/20 Cost 11.701424
Epoch   16/20 Cost 11.691505
Epoch   17/20 Cost 11.685121
Epoch   18/20 Cost 11.680006
Epoch   19/20 Cost 11.675381
Epoch   20/20 Cost 11.670943


## High-level Implementation with `nn.Module`

In [26]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)

In [39]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1):
    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 52013.953125
Epoch    1/20 Cost: 19244.888672
Epoch    2/20 Cost: 7129.252930
Epoch    3/20 Cost: 2649.758301
Epoch    4/20 Cost: 993.556519
Epoch    5/20 Cost: 381.205231
Epoch    6/20 Cost: 154.793320
Epoch    7/20 Cost: 71.074402
Epoch    8/20 Cost: 40.112957
Epoch    9/20 Cost: 28.657282
Epoch   10/20 Cost: 24.413631
Epoch   11/20 Cost: 22.836361
Epoch   12/20 Cost: 22.244961
Epoch   13/20 Cost: 22.018040
Epoch   14/20 Cost: 21.925928
Epoch   15/20 Cost: 21.883659
Epoch   16/20 Cost: 21.859797
Epoch   17/20 Cost: 21.842775
Epoch   18/20 Cost: 21.828262
Epoch   19/20 Cost: 21.814716
Epoch   20/20 Cost: 21.801517


## Dataset and DataLoader

<div class="alert alert-warning">
    pandas 기초지식이 필요할 것 같다
</div>

너무 데이터가 크면 `x_data`, `y_data` 를 전부 다 가져오지 말고, 필요한 배치만 가져올 수 밖에 없다.

[PyTorch Data Loading and Processing tutorial](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#iterating-through-the-dataset)